# Risk Sentiment - Equities

## Imports

In [19]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v4 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Data Cleaning

In [20]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_27655/711168680.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

In [21]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data /= 100
display(famafrench_data)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_27655/2041547973.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.000550,0.000188,0.000185,-0.000184,0.000189,0.000745,0.000695
1980-02-29,-0.000123,-0.000162,0.000059,-0.000095,0.000292,0.000789,-0.000132
1980-03-31,-0.001289,-0.000697,-0.000096,0.000182,-0.000105,-0.000958,-0.001181
1980-04-30,0.000396,0.000105,0.000103,-0.000218,0.000034,-0.000048,0.000574
1980-05-31,0.000526,0.000200,0.000038,0.000043,-0.000063,-0.000118,0.000618
...,...,...,...,...,...,...,...
2025-06-30,0.000486,-0.000002,-0.000160,-0.000320,0.000145,-0.000264,0.000527
2025-07-31,0.000198,-0.000015,-0.000127,-0.000029,-0.000208,-0.000096,0.000184
2025-08-31,0.000185,0.000488,0.000442,-0.000068,0.000207,-0.000354,0.000646


In [22]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)
display(benchmark_data)
benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()

benchmark_TR = benchmark_data[['MSCI World']].pct_change()
benchmark_XR = benchmark_TR.sub(riskfree, axis = 0)

benchmark_TR.columns = ['Benchmark Total Return']
benchmark_XR.columns = ['Benchmark Excess Return']
benchmark_returns = pd.concat([benchmark_TR, benchmark_XR], axis = 1).dropna()

display(benchmark_returns)

,S&P 500,MSCI World,FTSE WGBI,60/40
Date,,,,
1986-12-31,242.1700,NaN,156.5737,14.640989
1987-01-30,274.7800,NaN,161.2733,15.080442
1987-02-27,285.6377,NaN,163.7959,15.316327
1987-03-31,293.8792,NaN,167.5858,15.670715
1987-04-30,291.2698,NaN,169.5120,15.850831
...,...,...,...,...
2025-07-31,14412.5527,8214.1572,899.0753,418.585408
2025-08-29,14704.7217,8431.0801,911.3233,428.564691
2025-09-30,15240.0381,8705.7139,916.8774,440.268260


,Benchmark Total Return,Benchmark Excess Return
1991-01-31,0.034364,0.029164
1991-02-28,0.090450,0.085650
1991-03-31,-0.031331,-0.035731
1991-04-30,0.005910,0.000610
1991-05-31,0.020740,0.016040
...,...,...
2025-06-30,0.043488,0.040088
2025-07-31,0.013121,0.009721
2025-08-31,0.026408,0.022608
2025-09-30,0.032574,0.029274


In [23]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    f"{path}/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

display(equity_prices)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,1206.821289,NaN,NaN,7.33429,7487.55371,NaN,NaN
1997-10-31,1168.258667,NaN,NaN,6.82642,7084.46436,NaN,NaN
1997-11-30,1207.453491,NaN,NaN,6.34016,7152.55615,NaN,NaN
1997-12-31,1223.840210,NaN,NaN,5.86087,7365.37451,NaN,NaN
1998-01-31,1234.778442,NaN,NaN,6.36481,7690.75049,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,6308.888672,5641.31592,14994.83984,19.64406,12121.12598,14955.800000,6276.48438
2025-07-31,6430.451172,5627.64307,14552.98438,19.41757,12131.93066,15094.600000,6109.94238
2025-08-31,6529.819336,5881.38037,15274.74121,20.75129,12521.77637,15223.700000,6284.96094


## Global Variables

In [24]:
frequency = 1
t_cost = 0
# window = 12*20
short = True
beta_neutral = False
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [25]:
# --- Compute Risk Sentiment Signal ---
risk_sentiment_signal = equity_prices.pct_change(12).resample('ME').last()

display('Risk Sentiment Signal:')
display(risk_sentiment_signal)

'Risk Sentiment Signal:'

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.089128,0.068200,0.135337,0.154598,0.144851,0.289415,0.182202
2025-07-31,0.102824,0.049903,0.052154,0.073431,0.108819,0.101297,0.147454
2025-08-31,0.099490,0.061966,0.057742,0.147609,0.115843,0.169668,0.137625


In [26]:
# --- Composite Signal Construction ---
# Logic: For Equities, we want increasing Risk Sentiment (+)
risk_sentiment_signal = risk_sentiment_signal

display("Risk Sentiment Signal:")
display(risk_sentiment_signal)

'Risk Sentiment Signal:'

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.089128,0.068200,0.135337,0.154598,0.144851,0.289415,0.182202
2025-07-31,0.102824,0.049903,0.052154,0.073431,0.108819,0.101297,0.147454
2025-08-31,0.099490,0.061966,0.057742,0.147609,0.115843,0.169668,0.137625


## Asset Returns

In [27]:
# --- Equity Returns ---
equity_XR = equity_prices.pct_change()
equity_XR = equity_XR.resample('ME').last()

display(equity_XR)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


## Portfolio Construction

In [28]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

rank = risk_sentiment_signal.rank(axis = 1, method = 'average', ascending = False)
display(rank)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,6.0,7.0,5.0,3.0,4.0,1.0,2.0
2025-07-31,3.0,7.0,6.0,5.0,2.0,4.0,1.0
2025-08-31,5.0,6.0,7.0,2.0,4.0,1.0,3.0


In [29]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

rank_mean = rank.mean(axis = 1)
rank_stds = rank.std(axis = 1)
standardized_weights = -1 * rank.sub(rank_mean, axis = 0).div(rank_stds, axis = 0)

display(standardized_weights)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,-0.92582,-1.38873,-0.46291,0.46291,-0.00000,1.38873,0.92582
2025-07-31,0.46291,-1.38873,-0.92582,-0.46291,0.92582,-0.00000,1.38873
2025-08-31,-0.46291,-0.92582,-1.38873,0.92582,-0.00000,1.38873,0.46291


In [30]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret = (standardized_weights.shift(1) * equity_XR).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol = strategy_raw_ret.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor = target_vol / expected_vol.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights = standardized_weights.mul(lev_factor, axis=0).fillna(0).loc['1998-01-31':'2025-10-31']

display(final_strategy_weights)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1998-01-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-02-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-03-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-04-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-05-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
2025-06-30,-0.258529,-0.387794,-0.129265,0.129265,-0.000000,0.387794,0.258529
2025-07-31,0.128469,-0.385408,-0.256938,-0.128469,0.256938,-0.000000,0.385408
2025-08-31,-0.131010,-0.262021,-0.393031,0.262021,-0.000000,0.393031,0.131010


# Backtest

In [31]:
benchmark_TR = benchmark_TR.squeeze()
benchmark_XR = benchmark_XR.squeeze()

In [32]:
results = pmp.run_cc_strategy(
    weights = final_strategy_weights,
    returns = equity_XR,
    rf = riskfree,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark_XR,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_AU,w_CH,w_JP,w_UK,w_EM,w_EU
Date,,,,,,,,,,,,,
1998-02-28,0.000000,0.000000,0.064065,0.000000,0.0,0.0039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-03-31,0.000000,0.000000,0.039375,0.000000,0.0,0.0039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-04-30,0.000000,0.000000,0.005630,0.000000,0.0,0.0043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-05-31,0.000000,0.000000,-0.015149,0.000000,0.0,0.0040,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1998-06-30,0.000000,0.000000,0.020679,0.000000,0.0,0.0041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.010509,0.010509,0.040088,0.406828,0.0,0.0034,-0.337065,-0.498187,0.320240,0.160582,-0.164749,0.519178,0.000000
2025-07-31,-0.004984,-0.004984,0.009721,0.612206,0.0,0.0034,-0.339675,-0.499171,-0.161154,0.166300,0.000000,0.507189,0.326511
2025-08-31,-0.017663,-0.017663,0.022608,0.897743,0.0,0.0038,0.166554,-0.499222,-0.333036,-0.167742,0.334785,0.000000,0.498661


# Performance Statistics

In [33]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,6.9887,-0.8338
Arithm Avg Xs Return,4.9498,-2.8727
Std Xs Returns,15.5470,10.9340
Sharpe Arithmetic,0.3184,-0.2627
Geom Avg Total Return,5.9249,-1.4291
Geom Avg Xs Return,3.8686,-3.4854
Sharpe Geometric,0.2488,-0.3188
Min Xs Return,-19.0940,-14.2614
Max Xs Return,12.8084,11.4155
Skewness,-0.6049,-0.1811


In [ ]:
factor_data = factor_data[["MKT-RF", "SMB", "HML", "UMD", "BAB"]]